In [ ]:
from astropy.table import Table, vstack
from astropy.io import fits
import seaborn as sns
import glob
import matplotlib.pyplot as plt
from python_modules.muonpipe import usefull_func
import numpy as np
import pandas as pd
from python_modules.muonpipe.usefull_func import hist_wo_outliers, atmo_thick

size_outside = 400 # phe

In [ ]:
filepath = '/Users/vdk/muons2024/cscs_simulations/muon_table_run2.fits'
dat = Table.read(filepath)
df = dat.to_pandas()
df_good_data = df[df['good_ring']]

df_good_data.columns
np.mean(df_good_data['muon_efficiency'])

In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/cscs_simulations/tables_large_lst_muons/*')
listdir2 = glob.glob('/Users/vdk/muons2024/cscs_simulations/muons_zenith0/*')
# Muons analyzed using GlobalPeakWindowSum
listdir=listdir+listdir2
dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    dat = vstack([dat, dat2])

        
df = dat.to_pandas()
df_good_data = df[df['good_ring']]
np.mean(df_good_data['muon_efficiency'])

In [ ]:
new_df = df_good_data[df_good_data['muon_efficiency'] < 1]
new_df

In [ ]:
hist_wo_outliers(new_df['muon_efficiency'], outlier_value=5)

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(new_df['impact_x_array'], new_df['impact_y_array'], alpha = 0.1, s = 1, marker = 'x')
plt.xlabel("Impact parameter X [m]")
plt.ylabel("Impact parameter Y [m]")
plt.title("Reconstructed impact points of the muons on the LST mirror, transparency of dots = 0.1, 4e5 events")
plt.savefig('/Users/vdk/muons2024/images/plots_for_talk/ImpactParameter_Simulations_zenith0.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
mean_opt_eff = np.mean(df_good_data['muon_efficiency'])
mean_ring_width = np.mean(df_good_data['ring_width'])
mean_ring_radius = np.mean(df_good_data['ring_radius'])
mean_ring_size = np.mean(df_good_data['ring_size'])
mean_impact_parameter = np.mean(df_good_data['impact_parameter'])
print(np.mean(df_good_data['muon_efficiency']))
print(np.mean(df_good_data['ring_width']))
print(mean_ring_radius)

In [ ]:
df_good_data.columns

In [ ]:
param = 'ring_completeness'
print(f"Min value of {param} = {min(df_good_data[param])}")
print(f"Max value of {param} = {max(df_good_data[param])}")

In [ ]:
x_value_str = 'ring_width'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon efficiency', s = 20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_opt_eff, color='g', linestyle='--', label = f'Mean optical efficiency for this period = {round(mean_opt_eff, 3)}')
plt.axvline(x=mean_ring_width, color='orange', linestyle='--', label = f'Mean ring width for this period = {round(mean_ring_width,3)} deg')
plt.legend()
plt.ylim(0,1.42)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/WidthVsEfficiency_29-23Y_ylim2.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'ring_size'
y_value_str = 'muon_efficiency'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Muon efficiency', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_opt_eff, color='g', linestyle='--', label = f'Mean optical efficiency for this period = {round(mean_opt_eff, 3)}')
plt.axvline(x=mean_ring_size, color='orange', linestyle='--', label = f'Mean ring size for this period = {round(mean_ring_size,3)} pe')
plt.legend()
plt.ylim(0,1)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/SizeEfficiency_19-23Y_ylim2.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_width'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Ring Width', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_ring_width, color='g', linestyle='--', label = f'Mean width for this period = {round(mean_ring_width, 3)}')
plt.axvline(x=mean_ring_radius, color='orange', linestyle='--', label = f'Mean ring radius for this period = {round(mean_ring_radius,3)} pe')
plt.legend()
plt.ylim(0,0.2)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/RadiusVsWidth_19-23Y.png', dpi=200, format='png', bbox_inches='tight')

In [ ]:
x_value_str = 'ring_radius'
y_value_str = 'ring_size'

# Step 1: Create bins for 'ring_radius'
df_good_data[f'{x_value_str}_bin'] = pd.cut(df_good_data[x_value_str], bins=500)

# Step 2: Group by the new bin column and calculate mean 'ring_size' for each bin
binned_data = df_good_data.groupby(f'{x_value_str}_bin')[y_value_str].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data[f'{x_value_str}_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data[y_value_str], label='Binned Ring Size', s=20)

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data[x_value_str], df_good_data[y_value_str], 1)
plt.plot(df_good_data[x_value_str], m*df_good_data[x_value_str] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel(f'{x_value_str}')
plt.ylabel(f'{y_value_str}')
plt.axhline(y=mean_ring_size, color='g', linestyle='--', label = f'Mean ring_size for this period = {round(mean_ring_size, 3)}')
plt.axvline(x=mean_ring_radius, color='orange', linestyle='--', label = f'Mean ring radius for this period = {round(mean_ring_radius,3)} pe')
plt.legend()
plt.ylim(0,5000)
#plt.xlim(0,0.3)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/home/jovyan/XImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/home/jovyan/OptEffVSImpact.png', dpi=300, format='png', bbox_inches='tight')
#plt.savefig('/Users/vdk/muons2024/images/RadiusVsSize_19-23Y.png', dpi=200, format='png', bbox_inches='tight')